# [CSC8101] Engineering for AI - 2024 Spark Coursework

## Coursework overview

### Inputs

- **NYC Taxi Trips dataset** - list of recorded taxi trips, each with several characteristics, namely: distance, number of passengers, origin zone, destination zone and trip cost (total amount charged to customer).
- **NYC Zones dataset** - list of zones wherein trips can originate/terminate.

### Tasks

1. Data cleaning
  1. Remove "0 distance" and 'no passengers' records.
  2. Remove outlier records. 
2. Add new columns
  1. Join with zones dataset
  2. Compute the unit profitability of each trip
3. Zone summarisation and ranking
  1. Summarise trip data per zone
  2. Obtain the top 10 ranks according to:
    1. The total trip volume
    2. Their average profitabilitiy
    3. The total passenger volume
4. Record the total and task-specific execution times for each dataset size and format.

### How to

###### Code structure and implementation

- You must implement your solution to each task in the provided function code skeleton.
- The task-specific functions are combined together to form the full pipeline code, executed last (do not modify this code).
- Before implementing the specified function skeleton, you should develop and test your solution on separate code cells (create and destroy cells as needed).

###### Development

- Develop an initial working solution for the 'S' dataset and only then optimise it for larger dataset sizes.
- To perform vectorised operations on a DataFrame:
  - use the API docs to look for existing vectorised functions in: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html
  - actions to get around the lazy execution of spark: 
  https://spark.apache.org/docs/latest/rdd-programming-guide.html#actions
  - if a customised function is required (e.g. to add a new column based on a linear combination of other columns), implement your own User Defined Function (UDF). See:  https://spark.apache.org/docs/latest/sql-ref-functions-udf-scalar.html
- Use only the `pyspark.sql` API - documentation link below - (note that searching through the docs returns results from the `pyspark.sql` API together with the `pyspark.pandas` API):
  - https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.sql.html
- Periodically download your notebook to your computer as backup and safety measure against accidental file deletion.
 
###### Execution time measurement

- Execution time is calculated and returned by the Spark Engine and shown in the output region of the cell.
- To measure the execution time of a task you must perform a `collect` or similar operation (e.g. `take`) on the returned DataFrame.

## Task 0 - Read data

The code below is ready to run. **Do not modify this code**. It does the following:

- Reads the 'zones' dataset into variable 'zone_names'
- Defines the `init_trips` function that allows you to read the 'trips' dataset (from the DBFS FileStore) given the dataset size ('S' to 'XXL') and format ('parquet' or 'delta') as function arguments
- Defines the `pipeline` function, called in Task 4 to measure the execution time of the entire data processing pipeline
- Shows you how to call the `init_trips` function and display dataset characteristics (number of rows, schema)

In [0]:
## global imports
import pyspark.sql as ps
import pyspark.sql.functions as pf
import pandas as pd

# Load zone names dataset - (much faster to read small file from git than dbfs)
zones_file_url = 'https://raw.githubusercontent.com/mutazb999/CSC8101-lab-and-coursework/main/02-assignment-spark/taxi_zone_names.csv'
zone_names = spark.createDataFrame(pd.read_csv(zones_file_url))


In [0]:
# Function to load trips dataset by selected dataset size
def init_trips(size = 'S', data_format = "parquet", taxi_folder = "/FileStore/tables/taxi"):     
    
    files = {
        'S'  : ['2021_07'],
        'M'  : ['2021'],
        'L'  : ['2020_21'],
        'XL' : ['1_6_2019', '7_12_2019'],
        'XXL': ['1_6_2019', '7_12_2019', '2020_21']
    }
    
    # validate input dataset size
    if size not in files.keys():
        print("Invalid input dataset size. Must be one of {}".format(list(files.keys())))
        return None               
    
    if data_format == "parquet":
        filenames = list(map(lambda s: f'{taxi_folder}/parquet/tripdata_{s}.parquet', files[size]))
        trips_df = spark.read.parquet(filenames[0])
        
        for name in filenames[1:]:
            trips_df = trips_df.union(spark.read.parquet(name))
            
    elif data_format == "delta":
        filenames = f"{taxi_folder}/delta/taxi-{size}-delta/"
        trips_df = spark.read.format("delta").load(filenames)
    
    else:
        print("Invalid data format. Must be one of {}".format(['parquet', 'delta']))
        return None
        
    print(
    """
    Trips dataset loaded!
    ---
      Size: {s}
      Format: {f}
      Tables loaded: {ds}
      Number of trips (dataset rows): {tc:,}
    """.format(s = size, f = data_format, ds = filenames, tc = trips_df.count()))
    
    return trips_df

In [0]:
# helper function to print dataset row count
def print_count(df):
    print("Row count: {t:,}".format(t = df.count()))

def pipeline(trips_df, with_task_12 = False, zones_df = zone_names):
    # Do not edit
    #---

    ## Task 1.1
    _trips_11 = t11_remove_zeros(trips_df)

    ## Task 1.2
    if with_task_12:
        _trips_12 = t12_remove_outliers(_trips_11)
    else:
        _trips_12 = _trips_11

    ## Task 2.1
    _trips_21 = t21_join_zones(_trips_12, zones_df = zone_names)

    ## Task 2.2
    _trips_22 = t22_calc_profit(_trips_21)

    ## Task 3.1
    _graph = t31_summarise_trips(_trips_22)

    ## Task 3.2
    _zones = t32_summarise_zones_pairs(_graph)

    _top10_trips     = t32_top10_trips(_zones)
    _top10_profit    = t32_top10_profit(_zones)
    _top10_passenger = t32_top10_passenger(_zones)
    
    return([_top10_trips, _top10_profit, _top10_passenger])

In [0]:
# CHANGE the value of argument 'size' to record the pipeline execution times for increasing dataset sizes
SIZE = 'S'
DATA_FORMAT = 'parquet'

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)

# uncomment line only for small datasets
# trips.take(1)


    Trips dataset loaded!
    ---
      Size: S
      Format: parquet
      Tables loaded: ['/FileStore/tables/taxi/parquet/tripdata_2021_07.parquet']
      Number of trips (dataset rows): 2,898,033
    


In [0]:
print_count(trips)

Row count: 2,898,033


In [0]:
# dataset schemas
trips.printSchema()

root
 |-- index: long (nullable = true)
 |-- VendorID: double (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- cab_type: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |--

In [0]:
display(trips[['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']].take(5))

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
90,68,0.8,1.0,8.8
113,90,0.9,1.0,8.8
88,232,2.8,1.0,13.8
79,249,1.4,1.0,12.3
142,238,2.0,0.0,12.3


In [0]:
zone_names.printSchema()

root
 |-- LocationID: long (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [0]:
display(zone_names.take(5))

LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone


## Task 1 - Filter rows

**Input:** trips dataset

### Task 1.1 - Remove "0 distance" and 'no passengers' records

Remove dataset rows that represent invalid trips:

- Trips where `trip_distance == 0` (no distance travelled)
- Trips where `passenger_count == 0` and `total_amount == 0` (we want to retain records where `total_amount` > 0 - these may be significant as the taxi may have carried some parcel, for example)

Altogether, a record is removed if it satisfies the following conditions:

`trip_distance == 0` or `(passenger_count == 0` and `total_amount == 0)`.

**Recommended:** Select only the relevant dataset columns for this and subsequent tasks: `['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')]`

### Task 1.2 - Remove outliers using the modified z-score

Despite having removed spurious "zero passengers" trips in task 1.1, columns `total_amount` and `trip_distance` contain additional outlier values that must be identified and removed.

To identify and remove outliers, you will use the modified [z-score](https://en.wikipedia.org/wiki/Standard_score) method.
The modified z-score uses the median and [Median Absolute Deviation](https://en.wikipedia.org/wiki/Median_absolute_deviation) (MAD), instead of the mean and standard deviation, to determine how far an observation (indexed by i) is from the mean:

$$z_i = \frac{x_i - \mathit{median}(\mathbf{x})}{\mathbf{MAD}},$$

where x represents the input vector, xi is an element of x and zi is its corresponding z-score. In turn, the MAD formula is:

$$\mathbf{MAD} = 1.483 * \mathit{median}(\big\lvert x_i - \mathit{median}(\mathbf{x})\big\rvert).$$

Observations with **high** (absolute) z-score are considered outlier observations. A score is considered **high** if its __absolute z-score__ is larger than a threshold T = 3.5:

$$\big\lvert z_i \big\rvert > 3.5.$$

where T represents the number of unit standard deviations beyond which a score is considered an outlier ([wiki](https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule)).

This process is repeated twice, once for each of the columns `total_amount` and `trip_distance` (in any order).

**Important:** Use the surrogate function [`percentile_approx`](https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.functions.percentile_approx.html?highlight=percentile#pyspark.sql.functions.percentile_approx) to estimate the median (calculating the median values for a column is expensive as it cannot be parallelised efficiently).

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below

In [0]:
# Your solution implementation to task 1.1 goes HERE
def t11_remove_zeros(df):
    df = df.filter(
        (df["trip_distance"] != 0) | (df["passenger_count"] != 0) & (df["total_amount"] != 0)
    ).select(['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount'])
    return df

In [0]:
# execute task 1.1
trips_11 = t11_remove_zeros(trips)

print_count(trips_11)

## uncomment only for smaller datasets
display(trips_11.take(10))

Row count: 2,890,208


PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
90,68,0.8,1.0,8.8
113,90,0.9,1.0,8.8
88,232,2.8,1.0,13.8
79,249,1.4,1.0,12.3
142,238,2.0,0.0,12.3
114,90,1.6,1.0,12.8
90,144,1.8,1.0,13.3
114,48,2.0,1.0,14.75
48,152,5.7,1.0,22.3
234,148,1.8,1.0,14.75


In [0]:
# Task 1.2 - Remove outliers using the modified z-score

from pyspark.sql.functions import col, abs, expr

def t12_remove_outliers(df):
    columns = ['total_amount', 'trip_distance']
    
    for column in columns:
        median = df.agg(expr(f"percentile_approx({column}, 0.5)")).collect()[0][0]

        mad = df.withColumn('mad', abs(col(column) - median))\
                .agg(expr(f"percentile_approx(mad, 0.5)"))\
                .collect()[0][0] * 1.483

        if mad == 0:
            continue

        # Apply the filter with the z-score calculation
        df = df.filter(
            abs((col(column) - median) / mad) <= 3.5
        )
    
    return df


In [0]:
# execute task 1.2
trips_12 = t12_remove_outliers(trips_11)

print_count(trips_12)
display(trips_12.take(10))

Row count: 2,463,077


PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
90,68,0.8,1.0,8.8
113,90,0.9,1.0,8.8
88,232,2.8,1.0,13.8
79,249,1.4,1.0,12.3
142,238,2.0,0.0,12.3
114,90,1.6,1.0,12.8
90,144,1.8,1.0,13.3
114,48,2.0,1.0,14.75
48,152,5.7,1.0,22.3
234,148,1.8,1.0,14.75


## Task 2 - Compute new columns

### Task 2.1 - Zone names

Obtain the **start** and **end** zone names of each trip by joining the `trips` and `zone_names` datasets (i.e. by using the `zone_names` dataset as lookup table).

**Note:** The columns containing the start and end zone ids of each trip are named `PULocationID` and `DOLocationID`, respectively.

### Task 2.2 - Unit profitability

Compute the column `unit_profitability = total_amount / trip_distance`.

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below

In [0]:
# Your solution implementation to task 2.1 goes HERE
from pyspark.sql.functions import broadcast

def t21_join_zones(df, zones_df):
    df = df.join(broadcast(zones_df.withColumnRenamed("LocationID", "PULocationID").withColumnRenamed("Zone", "PUZone")), "PULocationID", "left_outer")
    df = df.join(broadcast(zones_df.withColumnRenamed("LocationID", "DOLocationID").withColumnRenamed("Zone", "DOZone")), "DOLocationID", "left_outer")
    
    return df

In [0]:
# execute task 2.1
trips_21 = t21_join_zones(trips_12, zones_df = zone_names)

print_count(trips_21)
display(trips_21.take(10))

Row count: 2,463,077


DOLocationID,PULocationID,trip_distance,passenger_count,total_amount,Borough,PUZone,service_zone,Borough,DOZone,service_zone
68,90,0.8,1.0,8.8,Manhattan,Flatiron,Yellow Zone,Manhattan,East Chelsea,Yellow Zone
90,113,0.9,1.0,8.8,Manhattan,Greenwich Village North,Yellow Zone,Manhattan,Flatiron,Yellow Zone
232,88,2.8,1.0,13.8,Manhattan,Financial District South,Yellow Zone,Manhattan,Two Bridges/Seward Park,Yellow Zone
249,79,1.4,1.0,12.3,Manhattan,East Village,Yellow Zone,Manhattan,West Village,Yellow Zone
238,142,2.0,0.0,12.3,Manhattan,Lincoln Square East,Yellow Zone,Manhattan,Upper West Side North,Yellow Zone
90,114,1.6,1.0,12.8,Manhattan,Greenwich Village South,Yellow Zone,Manhattan,Flatiron,Yellow Zone
144,90,1.8,1.0,13.3,Manhattan,Flatiron,Yellow Zone,Manhattan,Little Italy/NoLiTa,Yellow Zone
48,114,2.0,1.0,14.75,Manhattan,Greenwich Village South,Yellow Zone,Manhattan,Clinton East,Yellow Zone
152,48,5.7,1.0,22.3,Manhattan,Clinton East,Yellow Zone,Manhattan,Manhattanville,Boro Zone
148,234,1.8,1.0,14.75,Manhattan,Union Sq,Yellow Zone,Manhattan,Lower East Side,Yellow Zone


In [0]:
# Your solution implementation to task 2.2 goes HERE
def t22_calc_profit(df):
    df = df.withColumn("unit_profitability", col("total_amount") / col("trip_distance"))
    
    return df


In [0]:
# execute task 2.2
trips_22 = t22_calc_profit(trips_21)

print_count(trips_22)
display(trips_22.take(10))

Row count: 2,463,077


DOLocationID,PULocationID,trip_distance,passenger_count,total_amount,Borough,PUZone,service_zone,Borough,DOZone,service_zone,unit_profitability
68,90,0.8,1.0,8.8,Manhattan,Flatiron,Yellow Zone,Manhattan,East Chelsea,Yellow Zone,11.0
90,113,0.9,1.0,8.8,Manhattan,Greenwich Village North,Yellow Zone,Manhattan,Flatiron,Yellow Zone,9.777777777777779
232,88,2.8,1.0,13.8,Manhattan,Financial District South,Yellow Zone,Manhattan,Two Bridges/Seward Park,Yellow Zone,4.928571428571429
249,79,1.4,1.0,12.3,Manhattan,East Village,Yellow Zone,Manhattan,West Village,Yellow Zone,8.785714285714286
238,142,2.0,0.0,12.3,Manhattan,Lincoln Square East,Yellow Zone,Manhattan,Upper West Side North,Yellow Zone,6.15
90,114,1.6,1.0,12.8,Manhattan,Greenwich Village South,Yellow Zone,Manhattan,Flatiron,Yellow Zone,8.0
144,90,1.8,1.0,13.3,Manhattan,Flatiron,Yellow Zone,Manhattan,Little Italy/NoLiTa,Yellow Zone,7.388888888888889
48,114,2.0,1.0,14.75,Manhattan,Greenwich Village South,Yellow Zone,Manhattan,Clinton East,Yellow Zone,7.375
152,48,5.7,1.0,22.3,Manhattan,Clinton East,Yellow Zone,Manhattan,Manhattanville,Boro Zone,3.912280701754386
148,234,1.8,1.0,14.75,Manhattan,Union Sq,Yellow Zone,Manhattan,Lower East Side,Yellow Zone,8.194444444444445


## Task 3: Rank zones by traffic, passenger volume and profitability

### 3.1 - Summarise interzonal travel

Build a graph data structure of zone-to-zone traffic, representing aggregated data about trips between any two zones. The graph will have one node for each zone and one edge connecting each pair of zones. In addition, edges contain aggregate information about all trips between those zones. 

For example, zones Z1 and Z2 are connected by *two* edges: edge Z1 --> Z2 carries aggregate data about all trips that originated in Z1 and ended in Z2, and edge Z2 --> Z1 carries aggregate data about all trips that originated in Z2 and ended in Z1.

The aggregate information of interzonal travel must include the following data:

- `average_unit_profit` - the average unit profitability (calculated as `mean(unit_profitability)`).
- `trips_count` -- the total number of recorded trips.
- `total_passengers` -- the total number of passenger across all trips (sum of `passenger_count`).

This graph can be represented as a new dataframe, with schema:

\[`PULocationID`, `DOLocationID`, `average_unit_profit`, `trips_count`, `total_passengers` \]

__hint__: the `groupby()` operator produces a `pyspark.sql.GroupedData` structure. You can then calculate multiple aggregations from this using `pyspark.sql.GroupedData.agg()`: 
- https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.groupby.html
- https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.GroupedData.agg.html

### Task 3.2 - Obtain top-10 zones

For each of the following measures, report the top-10 zones _using their plain names you dereferenced in the previous step, not the codes_. Note that this requires ranking the nodes in different orders. Specifically, you need to calculate the following further aggregations:

- the **total** number of trips originating from Z. This is simply the sum of `trips_count` over all outgoing edges for Z, i.e., edges of the form Z -> \*
- the **average** profitability of a zone. This is the average of all `average_unit_profit` over all *outgoing* edges from Z.
- The **total** passenger volume measured as the **sum** of `total_passengers` carried in trips that originate from Z

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below

In [0]:
## Your solution to task 3.1 goes HERE
from pyspark.sql.functions import mean, count, sum

def t31_summarise_trips(df):
    graph = df.groupBy("PULocationID", "DOLocationID").agg(
        mean("unit_profitability").alias("average_unit_profit"),
        count("*").alias("trips_count"),
        sum("passenger_count").alias("total_passengers")
    )
    return graph


In [0]:
# execute task 3.1
graph = t31_summarise_trips(trips_22)

print_count(graph)
display(graph.take(10))

Row count: 14,455


PULocationID,DOLocationID,average_unit_profit,trips_count,total_passengers
90,231,7.432778991914051,1123,1576.0
87,33,6.817160729833857,105,145.0
90,142,6.873745233124353,909,1279.0
114,100,7.480600705110231,466,707.0
234,144,9.131779529653814,1488,2187.0
148,262,4.89353554100773,239,356.0
246,249,8.820105729239454,1150,1713.0
170,179,5.138416672570028,80,103.0
48,232,5.478542842122207,329,474.0
142,144,5.799719215582247,343,482.0


In [0]:
# Your solution to task 3.2 goes HERE (implement each of the functions below)
def t32_summarise_zones_pairs(graph_df, zones_df = zone_names):
    # Join graph with zone names for pickup locations
    graph_with_zones = graph_df.join(broadcast(zones_df.withColumnRenamed("LocationID", "PULocationID").withColumnRenamed("Zone", "PUZone")), "PULocationID", "left_outer")
    
    # Aggregate by PUZone to get metrics for each zone
    zones_agg = graph_with_zones.groupBy("PUZone").agg(
        sum("trips_count").alias("total_trips"),
        mean("average_unit_profit").alias("average_profitability"),
        sum("total_passengers").alias("total_passengers")
    )

    return zones_agg


# Top 10 ranked zones by traffic (trip volume)
def t32_top10_trips(df_zones):
    return df_zones.orderBy(col("total_trips").desc()).take(10)


# Top 10 ranked zones by profit
def t32_top10_profit(df_zones):
    return df_zones.orderBy(col("average_profitability").desc()).take(10)

# Top 10 ranked zones by passengers
def t32_top10_passenger(df_zones):
    return df_zones.orderBy(col("total_passengers").desc()).take(10)


In [0]:
# execute task 3.2
zones = t32_summarise_zones_pairs(graph)

top10_trips     = t32_top10_trips(zones)
top10_profit    = t32_top10_profit(zones)
top10_passenger = t32_top10_passenger(zones)

In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
display(top10_trips)

PUZone,total_trips,average_profitability,total_passengers
Upper East Side South,121811,6.905632960330168,169800.0
Upper East Side North,103449,7.090884640881556,143579.0
Penn Station/Madison Sq West,96361,7.918537727903516,136625.0
Midtown Center,93275,7.6860915333769055,134263.0
Murray Hill,87914,7.757619426672339,123110.0
Midtown East,84533,7.45525586506237,118884.0
Lincoln Square East,80632,7.572146532069324,114275.0
Upper West Side South,74165,6.9206408211866135,105037.0
Clinton East,74123,8.940849065869886,106488.0
Lenox Hill West,73950,6.5210824289445934,103161.0


In [0]:
# use 'display()' return a pandas DataFrame for 'pretty' output
display(top10_profit)

PUZone,total_trips,average_profitability,total_passengers
Bay Terrace/Fort Totten,57,373.64352610622007,18.0
Baisley Park,355,106.86575379124723,153.0
Saint Michaels Cemetery/Woodside,300,103.12501378067475,366.0
Prospect Park,76,90.59317954859831,67.0
Van Cortlandt Village,231,59.391859123131994,93.0
Glen Oaks,27,48.41059594606823,12.0
Grymes Hill/Clifton,5,38.65,5.0
Crotona Park,8,36.84437137879828,6.0
Eastchester,142,35.01882481592064,19.0
Flatbush/Ditmas Park,719,35.000379842565565,181.0


In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
display(top10_passenger)

PUZone,total_trips,average_profitability,total_passengers
Upper East Side South,121811,6.905632960330168,169800.0
Upper East Side North,103449,7.090884640881556,143579.0
Penn Station/Madison Sq West,96361,7.918537727903516,136625.0
Midtown Center,93275,7.6860915333769055,134263.0
Murray Hill,87914,7.757619426672339,123110.0
Midtown East,84533,7.45525586506237,118884.0
Lincoln Square East,80632,7.572146532069324,114275.0
Clinton East,74123,8.940849065869886,106488.0
Upper West Side South,74165,6.9206408211866135,105037.0
Union Sq,73365,7.57410265222722,104832.0


## Task 4 - Record the pipeline's execution time

Record the execution time of:

1. the whole pipeline
2. the whole pipeline except task 1.2

on the two tables below, for all dataset sizes: `'S'`, `'M'`, `'L'`, `'XL'`, `'XXL'`, and data formats: `parquet` and `delta`.

Analyse the resulting execution times and comment on the effect of dataset size, dataset format and task complexity (with and without task 1.2) on pipeline performance.

In [0]:
# CHANGE the value of the following arguments to record the pipeline execution times for increasing dataset sizes
import time

sizes = ['S', 'M', 'L', 'XL', 'XXL']
formats = ['parquet', 'delta']
with_task_12_options = [True, False]

execution_times = {}

for data_format in formats:
    for size in sizes:
        for with_task_12 in with_task_12_options:
            # Start timer
            start_time = time.time()
            
            # Load the dataset
            trips = init_trips(size, data_format)
            
            # Execute the pipeline
            pipeline(trips, with_task_12=with_task_12)
            
            # Record end-time and calculate the duration of execution time
            end_time = time.time()
            duration = end_time - start_time
            
            # Print the execution time for each run
            key = (data_format, size, with_task_12)
            execution_times[key] = duration

            print(f"Data Format: {data_format}, Size: {size}, With Task 1.2: {with_task_12}, Duration: {duration} seconds")



    Trips dataset loaded!
    ---
      Size: S
      Format: parquet
      Tables loaded: ['/FileStore/tables/taxi/parquet/tripdata_2021_07.parquet']
      Number of trips (dataset rows): 2,898,033
    
Data Format: parquet, Size: S, With Task 1.2: True, Duration: 12.336172103881836 seconds

    Trips dataset loaded!
    ---
      Size: S
      Format: parquet
      Tables loaded: ['/FileStore/tables/taxi/parquet/tripdata_2021_07.parquet']
      Number of trips (dataset rows): 2,898,033
    
Data Format: parquet, Size: S, With Task 1.2: False, Duration: 6.020719289779663 seconds

    Trips dataset loaded!
    ---
      Size: M
      Format: parquet
      Tables loaded: ['/FileStore/tables/taxi/parquet/tripdata_2021.parquet']
      Number of trips (dataset rows): 15,571,166
    
Data Format: parquet, Size: M, With Task 1.2: True, Duration: 49.636245012283325 seconds

    Trips dataset loaded!
    ---
      Size: M
      Format: parquet
      Tables loaded: ['/FileStore/tables/taxi/par

_Table 1. Pipeline performance for `parquet` format.（left 2 decimals for time & speed）_

| metric                           | S    | M    | L    | XL   | XXL  |
|----------------------------------|------|------|------|------|------|
| rows (M)                         |  2.898033 |  15.571166 |  41.953716 |  90.443069 |  132.396785 |
| execution time (w/o 1.2) (sec)   | 6.02 | 23.79 | 58.14 | 72.64 | 78.67 |
| execution time (sec)             | 12.34| 49.64 | 136.25| 159.38| 167.07|
| sec / 1M records (w/o 1.2)       | 2.08 | 1.53  | 1.39  | 0.80  | 0.59  |
| sec / 1M records                 | 4.26 | 3.19  | 3.25  | 1.76  | 1.26  |

_Table 2. Pipeline performance for `delta` format.（left 2 decimals for time & speed）_

| metric                           | S    | M    | L    | XL   | XXL  |
|----------------------------------|------|------|------|------|------|
| rows (M)                         |  2.898033 |  15.571166 |  41.953716 |  90.443069 |  132.396785 |
| execution time (w/o 1.2) (sec)   | 5.34  | 8.17  | 8.26  | 14.18 | 18.48 |
| execution time (sec)             | 11.82 | 18.35 | 22.19 | 38.90 | 42.89 |
| sec / 1M records (w/o 1.2)       | 1.84  | 0.52  | 0.20  | 0.16  | 0.14  |
| sec / 1M records                 | 4.08  | 1.18  | 0.53  | 0.43  | 0.32  |

I ran it on my personal cluster.